In [17]:
import duckdb
import sqlite3
import pandas as pd

# 1. Hubungkan ke file DuckDB kamu
con_duck = duckdb.connect('database.duckdb')  # ganti nama file sesuai punya kamu

# 2. Buat file SQLite tujuan
con_sqlite = sqlite3.connect('database_baru.sqlite')  # nama file SQLite baru

# 3. Ambil semua tabel dari DuckDB
tables = con_duck.execute("SHOW TABLES").fetchall()
tables = [t[0] for t in tables]

# 4. Untuk tiap tabel, salin ke SQLite
for table in tables:
    print(f'Memproses tabel: {table}')
    df = con_duck.execute(f"SELECT * FROM {table}").df()
    df.to_sql(table, con_sqlite, if_exists='replace', index=False)

# 5. Tutup koneksi
con_duck.close()
con_sqlite.close()

print("✅ Konversi selesai!")

Memproses tabel: dim_card
Memproses tabel: dim_customer
Memproses tabel: dim_demographics
Memproses tabel: dim_time
Memproses tabel: fact_transactions
✅ Konversi selesai!


In [24]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('database_baru.sqlite')
cursor = conn.cursor()

def get_table_names(conn):
  cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
  return [row[0] for row in cursor.fetchall()]

table_names = get_table_names(conn)

for table_name in table_names:
  print(f"Table: {table_name}")
  query = f"SELECT * FROM {table_name} LIMIT 10"  # Membatasi hingga 10 baris
  df = pd.read_sql_query(query, conn)
  print(df)
  print("-" * 40)  # Pemisah antar tabel



Table: dim_card
   Card_Key Card_Category
0         1          Blue
1         2          Gold
2         3        Silver
3         4      Platinum
----------------------------------------
Table: dim_customer
   customer_key  CLIENTNUM     Attrition_Flag  Customer_Age  Months_on_book  \
0     768805383  768805383  Existing Customer            45              39   
1     818770008  818770008  Existing Customer            49              44   
2     713982108  713982108  Existing Customer            51              36   
3     769911858  769911858  Existing Customer            40              34   
4     709106358  709106358  Existing Customer            40              21   
5     713061558  713061558  Existing Customer            44              36   
6     810347208  810347208  Existing Customer            51              46   
7     818906208  818906208  Existing Customer            32              27   
8     710930508  710930508  Existing Customer            37              36   
9  

In [30]:
import sqlite3

# Koneksi ke database
conn = sqlite3.connect("database_baru.sqlite")

# Hapus view & table jika sudah ada
conn.execute("DROP VIEW IF EXISTS v_avg_utilization_by_card;")
conn.execute("DROP VIEW IF EXISTS v_total_trans_by_income;")
conn.execute("DROP VIEW IF EXISTS v_avg_contacts_by_attrition;")
conn.execute("DROP VIEW IF EXISTS v_trans_by_customer_status;")
conn.execute("DROP TABLE IF EXISTS dm_analysis;")
conn.execute("DROP TABLE IF EXISTS dm_operational;")
conn.commit()

# Buat TABEL dm_analysis
conn.execute("""
CREATE TABLE dm_analysis AS
SELECT
  f.Transaction_ID,
  f.Credit_Limit,
  f.Total_Trans_Amt,
  f.Total_Trans_Ct,
  f.Total_Revolving_Bal,
  f.Avg_Utilization_Ratio,
  t.Months_Inactive_12_mon,
  t.Contacts_Count_12_mon,
  c.Customer_Age,
  c.Months_on_book,
  d.Card_Category,
  g.Gender,
  g.Education_Level,
  g.Income_Category
FROM fact_transactions f
JOIN dim_time t ON f.Time_Key = t.Time_Key
JOIN dim_customer c ON f.customer_key = c.customer_key
JOIN dim_card d ON f.Card_Key = d.Card_Key
JOIN dim_demographics g ON f.demographic_key = g.demographic_key;
""")
conn.commit()

# View 1
conn.execute("""
CREATE VIEW v_avg_utilization_by_card AS
SELECT
  Card_Category,
  AVG(Avg_Utilization_Ratio) AS avg_util_ratio
FROM dm_analysis
GROUP BY Card_Category;
""")

# View 2
conn.execute("""
CREATE VIEW v_total_trans_by_income AS
SELECT
  Income_Category,
  SUM(Total_Trans_Amt) AS total_transaction
FROM dm_analysis
GROUP BY Income_Category;
""")
conn.commit()

# Buat TABEL dm_operational
conn.execute("""
CREATE TABLE dm_operational AS
SELECT
  f.Transaction_ID,
  f.Time_Key,
  f.customer_key,
  f.Total_Trans_Ct,
  f.Total_Trans_Amt,
  f.Avg_Open_To_Buy,
  c.Attrition_Flag,
  t.Months_Inactive_12_mon,
  t.Contacts_Count_12_mon
FROM fact_transactions f
JOIN dim_customer c ON f.customer_key = c.customer_key
JOIN dim_time t ON f.Time_Key = t.Time_Key;
""")
conn.commit()

# View 3
conn.execute("""
CREATE VIEW v_avg_contacts_by_attrition AS
SELECT
  Attrition_Flag,
  AVG(Contacts_Count_12_mon) AS avg_contacts
FROM dm_operational
GROUP BY Attrition_Flag;
""")

# View 4
conn.execute("""
CREATE VIEW v_trans_by_customer_status AS
SELECT
  Attrition_Flag,
  AVG(Total_Trans_Ct) AS avg_transactions
FROM dm_operational
GROUP BY Attrition_Flag;
""")
conn.commit()

# Tutup koneksi
conn.close()

In [31]:
# prompt: coba buka tampilkan tabel

import sqlite3
import pandas as pd

# Koneksi ke database
conn = sqlite3.connect("database_baru.sqlite")

# Fungsi untuk menampilkan tabel
def show_table(table_name):
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql_query(query, conn)
    print(df)
    print("-" * 40)

# Tampilkan isi tabel dm_analysis
print("Isi tabel dm_analysis:")
show_table("dm_analysis")


# Tampilkan isi tabel dm_operational
print("Isi tabel dm_operational:")
show_table("dm_operational")


#Tampilkan view v_avg_utilization_by_card
print("Isi view v_avg_utilization_by_card:")
show_table("v_avg_utilization_by_card")

#Tampilkan view v_total_trans_by_income
print("Isi view v_total_trans_by_income:")
show_table("v_total_trans_by_income")

#Tampilkan view v_avg_contacts_by_attrition
print("Isi view v_avg_contacts_by_attrition:")
show_table("v_avg_contacts_by_attrition")

#Tampilkan view v_trans_by_customer_status
print("Isi view v_trans_by_customer_status:")
show_table("v_trans_by_customer_status")

conn.close()


Isi tabel dm_analysis:
       Transaction_ID  Credit_Limit  Total_Trans_Amt  Total_Trans_Ct  \
0                   1       12691.0             1144              42   
1                   2        8256.0             1291              33   
2                   3        3418.0             1887              20   
3                   4        3313.0             1171              20   
4                   5        4716.0              816              28   
...               ...           ...              ...             ...   
10122           10123        4003.0            15476             117   
10123           10124        4277.0             8764              69   
10124           10125        5409.0            10291              60   
10125           10126        5281.0             8395              62   
10126           10127       10388.0            10294              61   

       Total_Revolving_Bal  Avg_Utilization_Ratio  Months_Inactive_12_mon  \
0                      777         